In [1]:
import laspy
import numpy as np

from pathlib import Path

In [2]:
folder = Path('./FORinstance_dataset')
extensions = ('.laz', '.las')
for_instance_files = sorted(
            [f for f in folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions],
            key=lambda f: f.name
        )

for i, path in enumerate(for_instance_files):
    ext = path.suffix.lower()
    file = laspy.read(path)

    min_x = np.min(file.x)
    min_y = np.min(file.y)
    min_z = np.min(file.z)

    max_x = np.max(file.x)
    max_y = np.max(file.y)
    
    print(f'Tamaño: ({(max_x - min_x):.2f}, {(max_y - min_y):.2f})')

    x = file.x - min_x
    y = file.y - min_y
    z = file.z - min_z
    
    mask = (file.classification != 3) & (file.classification != 0)  # Eliminamos puntos no clasificados o inválidos
    file.points = file.points[mask]

    semantic_labels = np.array(file.classification)
    semantic_labels = np.where(semantic_labels == 2, 1, semantic_labels)  # Colapsamos subclases innecesarias
    semantic_labels = np.where(semantic_labels == 6, 5, semantic_labels)
    
    class_labels = np.array([1, 4, 5])
    remap = np.zeros(class_labels.max() + 1, dtype=int)
    remap[class_labels] = np.arange(len(class_labels))
    semantic_labels = remap[semantic_labels]

    file.classification = semantic_labels
    file.write(f'./Corrected/FORinstance/plot_{i}{ext}')


Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (28.07, 31.62)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (34.39, 36.33)
Tamaño: (27.83, 27.83)
Tamaño: (32.24, 32.24)
Tamaño: (32.24, 32.23)
Tamaño: (32.24, 32.15)
Tamaño: (34.69, 35.43)
Tamaño: (27.83, 27.83)
Tamaño: (27.82, 27.83)
Tamaño: (27.83, 27.83)
Tamaño: (32.23, 32.24)
Tamaño: (27.83, 27.83)
Tamaño: (27.82, 27.83)
Tamaño: (32.24, 32.25)
Tamaño: (27.83, 27.83)
Tamaño: (27.83, 27.83)


In [3]:
start_idx = 28
folder = Path('./NIBIO_MLS')
extensions = ('.laz', '.las')
for_instance_files = sorted(
            [f for f in folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions],
            key=lambda f: f.name
        )

for i, path in enumerate(for_instance_files):
    ext = path.suffix.lower()
    file = laspy.read(path)

    min_x = np.min(file.x)
    min_y = np.min(file.y)
    min_z = np.min(file.z)

    max_x = np.max(file.x)
    max_y = np.max(file.y)
    
    print(f'Tamaño: ({(max_x - min_x):.2f}, {(max_y - min_y):.2f})')

    x = file.x - min_x
    y = file.y - min_y
    z = file.z - min_z

    semantic_labels = np.array(file.label)
    ids = np.array(file.treeID)
    semantic_labels = np.where((semantic_labels == 1) | (semantic_labels == 3), 0, semantic_labels)
    semantic_labels = np.where((semantic_labels == 2) & (ids == 0), 0, semantic_labels)
    semantic_labels = np.where((semantic_labels == 2) & (ids != 0), 2, semantic_labels)
    semantic_labels = np.where((semantic_labels == 4), 1, semantic_labels)

    file.classification = semantic_labels
    file.write(f'./Corrected/NIBIO_MLS/plot_{i + start_idx}{ext}')


Tamaño: (12.00, 12.00)
Tamaño: (13.44, 11.99)
Tamaño: (14.53, 13.81)
Tamaño: (12.34, 12.86)
Tamaño: (11.99, 12.71)
Tamaño: (16.18, 16.06)
Tamaño: (12.89, 13.20)
Tamaño: (13.38, 15.95)
Tamaño: (12.00, 12.71)
Tamaño: (12.00, 12.00)
Tamaño: (12.00, 12.91)
Tamaño: (12.00, 12.00)
Tamaño: (12.92, 13.50)
Tamaño: (12.85, 13.84)
Tamaño: (14.01, 12.00)
Tamaño: (14.74, 13.47)
Tamaño: (6.89, 4.98)
Tamaño: (8.13, 22.06)
Tamaño: (15.11, 6.90)
Tamaño: (14.55, 22.26)
Tamaño: (12.00, 12.11)
Tamaño: (11.99, 13.45)
Tamaño: (12.00, 12.01)
Tamaño: (13.16, 13.98)
Tamaño: (12.10, 12.00)
Tamaño: (12.75, 15.23)
Tamaño: (12.00, 15.23)
Tamaño: (13.90, 14.75)
Tamaño: (13.79, 12.00)
Tamaño: (12.62, 12.00)
Tamaño: (13.42, 12.00)
Tamaño: (12.00, 12.00)
Tamaño: (12.33, 13.82)
Tamaño: (12.11, 14.70)
Tamaño: (11.99, 13.83)
Tamaño: (12.00, 14.60)
Tamaño: (12.49, 12.00)
Tamaño: (12.61, 12.75)
Tamaño: (12.00, 12.00)
Tamaño: (12.47, 15.51)
Tamaño: (12.57, 12.00)
Tamaño: (13.96, 13.29)
Tamaño: (14.23, 12.00)
Tamaño: (14.10,